In [2]:
!pip install llama-index --upgrade
!pip install llama-index-core
!pip install llama-index-llms-openai
!pip install llama-index-llms-replicate
!pip install llama-index-embeddings-huggingface

  Using cached llama_index_embeddings_huggingface-0.1.1-py3-none-any.whl (8.0 kB)
  Using cached torch-2.2.0-cp310-cp310-manylinux1_x86_64.whl (755.5 MB)
  Using cached transformers-4.37.2-py3-none-any.whl (8.4 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Attempting uninstall: torch
    Found existing installation: torch 2.1.0+cu121
    Uninstalling torch-2.1.0+cu121:
      Successfully uninstalled torch-2.1.0+cu121
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tor

In [3]:
# install replicate client
import os
from google.colab import userdata
replicate_api_token = userdata.get('replicate_api_token')

In [4]:
!pip install pypdf
!wget https://admin.owners.infinitiusa.com/content/manualsandguides/IPL_G_Coupe/2011/2011-Infiniti-IPL-G-Coupe.pdf

--2024-02-19 03:19:33--  https://admin.owners.infinitiusa.com/content/manualsandguides/IPL_G_Coupe/2011/2011-Infiniti-IPL-G-Coupe.pdf
Resolving admin.owners.infinitiusa.com (admin.owners.infinitiusa.com)... 3.214.101.166, 35.169.218.39
Connecting to admin.owners.infinitiusa.com (admin.owners.infinitiusa.com)|3.214.101.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3971465 (3.8M) [application/pdf]
Saving to: ‘2011-Infiniti-IPL-G-Coupe.pdf’

2011-Infiniti-IPL-G 100%[===================>]   3.79M  --.-KB/s    in 0.06s   

2024-02-19 03:19:33 (67.6 MB/s) - ‘2011-Infiniti-IPL-G-Coupe.pdf’ saved [3971465/3971465]



In [5]:
import os
os.environ["REPLICATE_API_TOKEN"] = replicate_api_token

from llama_index.core import Settings, VectorStoreIndex, SimpleDirectoryReader
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.replicate import Replicate
from transformers import AutoTokenizer

# set the LLM
llama2_7b_chat = "meta/llama-2-7b-chat:8e6975e5ed6174911a6ff3d60540dfd4844201974602551e10e9e87ab143d81e"
Settings.llm = Replicate(
    model=llama2_7b_chat,
    temperature=0.01,
    additional_kwargs={"top_p": 1, "max_new_tokens": 300},
)
# set tokenizer to match LLM
Settings.tokenizer = AutoTokenizer.from_pretrained(
    "NousResearch/Llama-2-7b-chat-hf"
)
# set the embed model
Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)
# Read documents from the specified directory and load a specific document, "report.pdf".
documents = SimpleDirectoryReader("./").load_data("2011-Infiniti-IPL-G-Coupe.pdf")

# Create a VectorStoreIndex object from the documents. This will involve processing the documents
# and creating a vector representation for each of them, suitable for semantic searching.
index = VectorStoreIndex.from_documents(
    documents,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Loading files: 100%|██████████| 1/1 [00:15<00:00, 15.92s/file]


In [9]:
!pip install replicate

In [11]:
query_engine = index.as_query_engine()
response = query_engine.query("How do I clean car glass?")
print(f"{response.response}")

 To clean the car glass, follow these steps:
1. Lift the wiper arm away from the windshield.
2. Push the release tab and then move the wiper blade down the wiper arm while pushing the release tab to remove.
3. Insert the new wiper blade onto the wiper arm until a click sounds.
4. Rotate the wiper blade so the dimple is in the groove.
If your windshield is still not clear after cleaning the blades and using the wiper, replace the blades.
To maintain the appearance of your vehicle, it is important to take proper care of it. Wash your vehicle as soon as possible after a rainfall to prevent possible damage from acid rain. Store or park your vehicle inside a garage or covered area whenever possible. Whenever necessary, wash dirt off the vehicle with a wet sponge and plenty of water. Clean the vehicle thoroughly using a mild soap, special vehicle soap, or general purpose dishwashing liquid mixed with clean, lukewarm (never hot) water. Avoid using strong household soap, strong chemical deterg